In [ ]:
# Magic functions -- Run Once
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import matplotlib.pyplot as plt
# Set default figure size
plt.rcParams["figure.figsize"] = (12, 5)

import warnings
# Ignore all warnings (This might not be a good idea)
warnings.filterwarnings('ignore')

# Move up one folder to reach the repo root
%cd ..

from utils.notebook.generic import full_width_notebook
# Set notebook to 100% width
full_width_notebook()

In [ ]:
from main import parse_args_string, prepare_for_task
from utils.notebook.generic import notebook_input_prompt
from utils.visualization import show_tagged_scene, get_tagged_scene_table_legend
from utils.file import read_json
from runner import custom_question_inference

data_version_name = "CLEAR_50k_4_inst_audio"
model_folder = "CLEAR_50k_4_inst_audio_config_extractor_slim_interleaved_3_block_64_proj_40_epoch_876944_extractor_slim/2020-09-09_01h04"

model_restore_path = f"output_synced/training/{model_folder}"
data_root_path = "data"

# Read arguments from restored model
experiment_arguments = read_json(f"{model_restore_path}/arguments.json")
preprocessing_arguments = read_json(f"{model_restore_path}/preprocessed_arguments.json")
preprocessed_folder = experiment_arguments['preprocessed_folder_name']
config_path = experiment_arguments['config_path']
random_seed = experiment_arguments['random_seed']
#random_seed = 876944

# Options
use_cpu = False
do_transforms_online = True
use_training_data_stats = False    # If true, will normalize data using stats used to train the restored model
normalize_with_clear_stats = False
resample_to = None    # 24000
spectrogram_n_fft = 512
spectrogram_hop_length = 2048
spectrogram_keep_point = None #256
rgb_spectrogram = False
scene_sample_freq = 48000


# Model weight restore
model_weight_path = f"{model_restore_path}/best/model.pt.tar"
model_dict_file_path = f"{model_restore_path}/dict.json"

arguments = (
    f"--notebook_model_inference --version_name {data_version_name} --config_path {config_path} --batch_size 4 "
    f"--random_seed {random_seed} --dict_folder questions "
    f"--film_model_weight_path {model_weight_path} --dict_file_path {model_dict_file_path}{' --use_cpu' if use_cpu else ''}"
)

if not do_transforms_online:
    arguments += f' --h5_image_input --preprocessed_folder_name {preprocessed_folder}'
    max_frequency = preprocessed_arguments['resample_to']/2 if preprocessed_arguments['resample_to'] else scene_sample_freq/2
    
    if preprocessed_arguments['spectrogram_keep_freq_point']:
        max_frequency = int(max_frequency * (preprocessed_arguments['spectrogram_keep_freq_point']/preprocessed_arguments['spectrogram_n_fft']))
else:
    arguments += f' --spectrogram_n_fft {spectrogram_n_fft} --spectrogram_hop_length {spectrogram_hop_length}'
    if spectrogram_keep_point:
        arguments += f' --spectrogram_keep_freq_point {spectrogram_keep_point}'
        
    if normalize_with_clear_stats:
        arguments += ' --normalize_with_clear_stats'
        
    if rgb_spectrogram:
        arguments += ' --spectrogram_rgb'
        
    arguments += ' --normalize_zero_one --pad_to_largest_image'
    #arguments += ' --pad_to_largest_image'
    
    preprocessed_folder = "preprocessed_audio"
    
    if resample_to:
        preprocessed_folder += f"_resample_{resample_to}"
    
    preprocessed_folder += f"_win_{spectrogram_n_fft}_hop_{spectrogram_hop_length}"
    
    if spectrogram_keep_point:
        preprocessed_folder += f'_keep_{spectrogram_keep_point}'
        
    preprocessed_folder += '_norm_zero_one_norm'
    
    if normalize_with_clear_stats:
        preprocessed_folder += '_clear_stats'
        
    if rgb_spectrogram:
        preprocessed_folder += '_RGB'
    
    arguments += f' --preprocessed_folder_name {preprocessed_folder}'
    
    if use_training_data_stats:
        arguments += f' --clear_stats_file_path {model_restore_path}/preprocessed_data_stats.json'
        
    if not use_cpu:
        arguments += ' --do_transforms_on_gpu'
        
    max_frequency = resample_to/2 if resample_to else scene_sample_freq/2
    
    if spectrogram_keep_point:
        max_frequency = int(max_frequency * (spectrogram_keep_point/spectrogram_n_fft))
            
args = parse_args_string(arguments)
task_and_more, dataloaders, model_and_more = prepare_for_task(args)
task, args, flags, paths, device = task_and_more
film_model_config, film_model, optimizer, loss_criterion, scheduler, tensorboard = model_and_more
datasets = {set_type:dloader.dataset for set_type, dloader in dataloaders.items()}

# Retrieve clear stats
import json
if use_training_data_stats:
    data_stats_filepath = f"{model_restore_path}/preprocessed_data_stats.json"
else:
    data_stats_filepath = f"{data_root_path}/{data_version_name}/{preprocessed_folder}/clear_stats.json"

exp_stats = read_json(f"{model_restore_path}/stats.json")
test_stats = read_json(f"{model_restore_path}/test_stats.json")

clear_stats = read_json(data_stats_filepath)
if 'normalize_zero_one' in arguments:
    max_val = clear_stats['max']
    min_val = clear_stats['min']

    clear_stats = {
        'mean': [(m - min_val)/(max_val - min_val) for m in clear_stats['mean']],
        'std': [(s - min_val)/(max_val - min_val) for s in clear_stats['std']]
    }

# Print model summary
with open(f"{model_restore_path}/model_summary.txt", 'r') as f:
    print(f.read())

print(f"Model was trained for {len(exp_stats)} epoch on {experiment_arguments['version_name']} -- {preprocessed_folder}")
print(f"Achieved {float(exp_stats[0]['train_acc'])*100:.2f}% on training set and {float(exp_stats[0]['val_acc'])*100:.2f}% on validation set")
print(f"Got {float(test_stats['accuracy'])*100:.2f}% on test set ({test_stats['version_name']})")


In [ ]:
# Retrieve random game
set_type = 'train'
random_game = datasets[set_type].get_random_game(return_game=True)
#random_game = datasets[set_type][6]
scene_id = random_game['scene_id']

In [ ]:
img = random_game['image'].cpu().squeeze(0)
plt.figure()
plt.imshow(img,cmap='viridis')

In [ ]:
print(arguments)

In [ ]:
datasets['train'].get_min_width_image_dims()

In [ ]:
random_game

In [ ]:
random_game['image'].squeeze(0).shape

In [ ]:
import matplotlib.pyplot as plt

cmap = plt.get_cmap('Blues')
colored = cmap(random_game['image'].squeeze(0).cpu(), alpha=None)[:,:,:3]
#plt.figure()
#plt.imshow(random_game['image'].squeeze(0).cpu())
plt.figure()
plt.hist(colored[:,:,0].reshape(1,-1))
#colored[:,:,:3]

In [ ]:
print(random_game['image'].min(), random_game['image'].max(), random_game['image'].mean(), random_game['image'].std())

In [ ]:
# Show scene & Example questions
from utils.notebook.inference import show_game_notebook_input

custom_question, custom_questions, legend = show_game_notebook_input(dataloaders[set_type], random_game, clear_stats,
                                                                     remove_image_padding=False, max_frequency=max_frequency, fill_rect=False)

In [ ]:
from utils.notebook.inference import do_custom_question_inference, show_gradcam

# MODIFY THIS TO GET GRADCAM FOR DIFFERENT GUESSES
gradcam_guess_id = 0

custom_game, top_preds = do_custom_question_inference(device, film_model, dataloaders[set_type], custom_question, scene_id, nb_top_pred=5)
display(legend)

In [ ]:
layers_to_observe = {
    #'stem_conv': film_model.stem_conv,
    #'resblocks[0].conv1': film_model.resblocks[0].conv1,
    #'resblocks[0].conv2': film_model.resblocks[0].conv2,
    #'resblocks[0].film_layer': film_model.resblocks[0].film_layer,
    #'resblocks[0]': film_model.resblocks[0],
    #'resblocks[1].conv1': film_model.resblocks[1].conv1,
    #'resblocks[1].conv2': film_model.resblocks[1].conv2,
    #'resblocks[1].film_layer': film_model.resblocks[1].film_layer,
    #'resblocks[1]': film_model.resblocks[1],
    #'resblocks[2].conv1': film_model.resblocks[2].conv1,
    #'resblocks[2].conv2': film_model.resblocks[2].conv2,
    #'resblocks[2].film_layer': film_model.resblocks[2].film_layer,
    #'resblocks[2]': film_model.resblocks[2],
    #'resblocks[3].conv1': film_model.resblocks[3].conv1,
    #'resblocks[3].conv2': film_model.resblocks[3].conv2,
    #'resblocks[3].film_layer': film_model.resblocks[3].film_layer,
    #'resblocks[3]': film_model.resblocks[3],
    'classifier_conv': film_model.classifier.classif_conv
    #'classifier.logits': film_model.classifier.logits
}

display(legend)
heatmaps = show_gradcam(device, film_model, dataloaders[set_type], custom_game, scene_id, guess_id=gradcam_guess_id, 
                        top_preds=top_preds, clear_stats=clear_stats, apply_relu=True, target_layers=layers_to_observe)

In [ ]:
hook_handles = []

In [ ]:
from torch import nn

activations = {}
weights = {}
biases = {}

def forward_hook(name):
    def hook(module, inp, output):
        activations[name] = output
        if isinstance(module, nn.Sequential):
            for m in module:
                if isinstance(m, nn.Conv2d):
                    weights[name] = m.weight
                    
                    biases[name] = m.bias
        
    return hook

for layer_name, layer in layers_to_observe.items():
    hook_handles.append(layer.register_forward_hook(forward_hook(layer_name)))

In [ ]:
for hook in hook_handles:
    hook.remove()

In [ ]:
weights['stem_conv'].shape

In [ ]:
activations['stem_conv'].min()

In [ ]:
from torchvision.utils import make_grid
import numpy as np
def show(img):
    npimg = img.numpy()
    
    plt.figure()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    #plt.imshow(npimg, interpolation='nearest')
    
def separate_features(activations):
    bs, c, h, w = activations.size()
    assert bs == 1, "Only work for batch size of 1"
    # Remove batch size
    activations = activations.reshape((c,h,w))
    activations = activations.div(activations.max())
    return list(activations.split(1, dim=0))

#show(separate_features(activations['stem_conv'])[0])


normalized_activations = activations['classifier.logits'].sub(activations['classifier.logits'].min()).div(activations['classifier.logits'].max().sub(activations['classifier.logits'].min()))
separated_feature_maps = separate_features(activations['resblocks[3].film_layer'])
show(make_grid(separated_feature_maps[:32], nrow=10))
show(make_grid(separated_feature_maps[32:64], nrow=10))
show(make_grid(separated_feature_maps[64:96], nrow=10))
show(make_grid(separated_feature_maps[96:], nrow=10))

In [ ]:
separated_feature_maps = separate_features(activations['resblocks[3].conv2'])
show(make_grid(separated_feature_maps[:32], nrow=10))
show(make_grid(separated_feature_maps[32:64], nrow=10))
show(make_grid(separated_feature_maps[64:96], nrow=10))
show(make_grid(separated_feature_maps[96:], nrow=10))

In [ ]:
import torch.nn.functional as F

feature_map = F.upsample(separated_feature_maps[0].unsqueeze(0), size=random_game['image'].size()[1:], mode='bilinear', align_corners=False)

heatmap, result = visualize_cam(feature_map, random_game['image'])

In [ ]:
#show(heatmap)

img_min = random_game['image'].min()
normalized = random_game['image'].sub(img_min).div(random_game['image'].max() - img_min)

heatmaps = []
combined = []
for feature_map in separated_feature_maps:
    upsampled = F.upsample(feature_map.unsqueeze(0), size=random_game['image'].size()[1:], mode='bilinear', align_corners=False)
    up_min = upsampled.min()
    upsampled = upsampled.sub(up_min).div(upsampled.max() - up_min)
    heatmap, result = visualize_cam(upsampled, normalized)
    
    heatmaps.append(heatmap)
    combined.append(result)
    
show(make_grid(heatmaps))
show(make_grid(combined))

In [ ]:
print(heatmaps[28].mean())
show(heatmaps[28])

In [ ]:
plt.figure()
plt.imshow(separated_feature_maps[0].squeeze(0).numpy(), interpolation='nearest')


#show(separated_feature_maps[0])

In [ ]:
to_plot = []

for i, feature_map in enumerate(separated_feature_maps):
    silence_mean = feature_map[0,:, 10:].sum()
    rest_mean = feature_map[0,:, :10].sum()
    
    if silence_mean > rest_mean:
        print(i)
        to_plot.append(feature_map)

show(make_grid(to_plot, normalize=True))

In [ ]:
import torch
def visualize_cam(mask, img):
    """ Taken from https://github.com/vickyliin/gradcam_plus_plus-pytorch
    Make heatmap from mask and synthesize GradCAM result image using heatmap and img.
    Args:
        mask (torch.tensor): mask shape of (1, 1, H, W) and each element has value in range [0, 1]
        img (torch.tensor): img shape of (1, 3, H, W) and each pixel value is in range [0, 1]

    Return:
        heatmap (torch.tensor): heatmap img shape of (3, H, W)
        result (torch.tensor): synthesized GradCAM result of same shape with heatmap.
    """
    import cv2       # Not an official dependency
    heatmap = (255 * mask.squeeze()).type(torch.uint8).cpu().numpy()
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    heatmap = torch.from_numpy(heatmap).permute(2, 0, 1).float().div(255)
    b, g, r = heatmap.split(1)
    heatmap = torch.cat([r, g, b])

    result = heatmap*0.2+img.cpu()
    min_val = result.min()
    result = result.sub(min_val).div(result.max().sub(min_val)).squeeze()

    return heatmap, result